In [1]:
require 'nn'
require 'cutorch'
require 'cudnn'
require 'paths'
image = require 'image'
require 'xlua'        -- for progress bar
tsne = require 'tsne'
require 'Dataframe' -- imports many dependencies to read a csv file!

In [2]:
net = torch.load('./deepDress.AlexnetBaseline.t7')

In [3]:
print(tostring(net))

nn.Sequential {
  [input -> (1) -> (2) -> (3) -> (4) -> (5) -> (6) -> (7) -> (8) -> (9) -> (10) -> (11) -> (12) -> (13) -> (14) -> (15) -> (16) -> (17) -> (18) -> (19) -> (20) -> (21) -> (22) -> (23) -> (24) -> output]
  (1): cudnn.SpatialConvolution(3 -> 96, 11x11, 4,4)
  (2): cudnn.ReLU
  (3): cudnn.SpatialMaxPooling(3x3, 2,2)
  (4): cudnn.SpatialCrossMapLRN
  (5): cudnn.SpatialConvolution(96 -> 256, 5x5, 1,1, 2,2)
  (6): cudnn.ReLU
  (7): cudnn.SpatialMaxPooling(3x3, 2,2)
  (8): cudnn.SpatialCrossMapLRN
  (9): cudnn.SpatialConvolution(256 -> 384, 3x3, 1,1, 1,1)
  (10): cudnn.ReLU
  (11): cudnn.SpatialConvolution(384 -> 384, 3x3, 1,1, 1,1)
  (12): cudnn.ReLU
  (13): cudnn.SpatialConvolution(384 -> 256, 3x3, 1,1, 1,1)
  (14): cudnn.ReLU
  (15): cudnn.SpatialMaxPooling(3x3, 2,2)
  (16): nn.View(-1)
  (17): nn.Linear(9216 -> 4096)
  (18): cudnn.ReLU
  (19): nn.Dropout(0.500000)
  (20): nn.Linear(4096 -> 4096)
  (21): cudnn.ReLU
  (22): nn.Dropout(0.500000)
  (23): nn.Linear(4096 -> 2573

In [4]:
img_mean_name = 'ilsvrc_2012_mean.t7'

function preprocess(im, img_mean)
  -- rescale the image
  local im3 = image.scale(im,224,224,'bilinear')*255
  -- RGB2BGR
  local im4 = im3:clone()
  im4[{1,{},{}}] = im3[{3,{},{}}]
  im4[{3,{},{}}] = im3[{1,{},{}}]

  -- subtract imagenet mean
  return im4 - image.scale(img_mean, 224, 224, 'bilinear')
end

img_mean = torch.load(img_mean_name).img_mean:transpose(3,1)

In [5]:
-- Features path
-- step 1,read randomly from files

-- step 2, extract features
-- step 3, run t-sne from manifold

In [6]:
function extract_feat(image_name, layer_to_extract)
  im = image.load(image_name, 3)

  I = preprocess(im, img_mean)
  -- Propagate through the network and sort outputs in decreasing order and show 5 best classes

  -- do forward pass of model on the images
  net:forward(I:cuda()) -- super inefficient, should do batch
  
  -- read the activ/ations from the requested layer
  local feat = net.modules[layer_to_extract].output

  return feat
end

In [8]:
datadir = '/media/saurabh/my ssd/nordstrom/'
num_labels = #paths.dir(datadir) - 2 -- for '.', '..' filenames

In [9]:
feat = torch.CudaLongTensor(num_labels, 4096):zero()
labels = {}
print(feat:size())

 3802
 4096
[torch.LongStorage of size 2]



In [10]:
-- load features
local cntr = 1
for f in paths.files(datadir, 'jpg') do
    xlua.progress(cntr, num_labels) -- display progress
    local imgfeat = extract_feat(datadir .. f, 20)
    local test = imgfeat:float()
    labels[cntr]  = f
    feat[{cntr, {}}] = test
    cntr = cntr + 1
end

print('finished loading features for ' .. cntr .. ' products')

Progress: 44 / 3802	


Progress: 100 / 3802	


Progress: 151 / 3802	


Progress: 199 / 3802	


Progress: 248 / 3802	


...

Progress: 3625 / 3802	


Progress: 3676 / 3802	


Progress: 3719 / 3802	


Progress: 3769 / 3802	


finished loading features for 3803 products	


In [11]:
local timer = torch.Timer()
p = tsne(feat:double(), 2, 10, 5000, 0.5, 7)
print('Successfully performed t-SNE in ' .. timer:time().real .. ' seconds.')

Performing t-SNE using 7 cores.
Using no_dims = 2, perplexity = 10.000000, and theta = 0.500000
Computing input similarities...


Building tree...


Done in 20.00 seconds (sparsity = 0.013318)!
Learning embedding...


Iteration 50: error is 96.522656 (50 iterations in 1.00 seconds)


Iteration 100: error is 96.438898 (50 iterations in 0.00 seconds)


...

Iteration 4800: error is 2.059934 (50 iterations in 0.00 seconds)


Iteration 4850: error is 2.059106 (50 iterations in 0.00 seconds)


Iteration 4900: error is 2.058220 (50 iterations in 0.00 seconds)


Iteration 4950: error is 2.057539 (50 iterations in 1.00 seconds)


Iteration 4999: error is 2.056721 (50 iterations in 0.00 seconds)
Fitting performed in 28.00 seconds.
Successfully performed t-SNE in 48.861529111862 seconds.	


In [12]:
df = Dataframe()

In [13]:
df:load_csv{path='/home/saurabh/Documents/Code/python/nordstrom/nordstrom4.csv', header=true}


+-------------------------------------------------+
| seq | id      | color               | IsDefault |
+-------------------------------------------------+
|   1 | 5238439 | BLACK               | true      |
|   2 | 5238439 | BLUE                | false     |
|   3 | 5204158 | BLACK               | true      |
|   4 | 5204158 | NAVY                | false     |
|   5 | 5204158 | WINE RED            | false     |
|   6 | 5249813 | BLACK               | false     |
|   7 | 5249813 | DARK GREY           | true      |
|   8 | 5249813 | MERLOT              | false     |
|   9 | 5249813 | OLIVE               | false     |
|  10 | 5160873 | GREY CLOUDY HEATHER | true      |
| ...                                             |
+-------------------------------------------------+
	


In [14]:
df:where('seq', tonumber('63')):get_column('id')[1]

5244867	


In [15]:
-- write to file
--local s = {900,1200}
local s = {1800, 2400}
local b = 32

local p_ = p:clone()
local px = p_:select(2,1)
local py = p_:select(2,2)

px = (px - px:min()) / (px:max() - px:min())
py = (py - py:min()) / (py:max() - py:min())

local f = assert(io.open('tsne.nordstrom.csv', 'w'))
f:write('eigen1,eigen2,stylename,img,png\n')

-- im = torch.FloatTensor(3,s[1],s[2]):fill(0)
im = torch.DoubleTensor(3,s[1],s[2]):fill(0)
for idx, c in pairs(labels)
do
    xlua.progress(idx, num_labels) -- display progress
    local localfile = datadir .. c
    local seq, ext = string.gsub(c, '.jpg', '')
    local x = math.ceil(b + px[idx]*s[1]*0.95)
    local y = math.ceil(b + py[idx]*s[2]*0.95)
    if paths.filep(localfile) then
        local img = image.load(localfile, 3)
        local scaled = image.scale(img,b,b)
        local buff = x .. ',' .. 
            y .. ',' .. 
            df:where('seq', tonumber(seq)):get_column('id')[1] .. ',' .. -- ugh, probably slow
            c .. ',' ..
            seq .. '.png' ..
--        ',' .. 
--            products[c]['imageURL'] .. ',' ..
--            products[c]['designer'] .. 
            '\n'

        f:write(buff)
        im[{{},{x-b/2,x+b/2-1},{y-b/2,y+b/2-1}}]:add(scaled)
    end
end

f:close()
print('finished writing csv file')

Progress: 1 / 3802	


Progress: 433 / 3802	


Progress: 847 / 3802	


Progress: 1273 / 3802	


Progress: 1696 / 3802	


Progress: 2132 / 3802	


Progress: 2563 / 3802	


Progress: 2999 / 3802	


Progress: 3432 / 3802	


finished writing csv file	


In [16]:
itorch.image(-im)